In [1]:
from pydriller import Repository
import csv
import os.path
import pandas as pd
from jira import JIRA

In [2]:
repoPath = 'https://github.com/apache/ActiveMQ'
commitsFile = 'commits.csv'
commits = []

if not(os.path.isfile(commitsFile)) :
    with open(commitsFile, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['hash', 'msg', 'insertions'])
        writer.writeheader()
        
        for commit in Repository(repoPath).traverse_commits():
            if commit.deletions == 0 and commit.insertions > 0 : #commits which only insert new lines
                record = {'hash': commit.hash, 'msg': commit.msg, 'insertions': commit.insertions}
                writer.writerow(record)

In [3]:
jira = JIRA(server="https://issues.apache.org/jira")

def getAllIssues(jiraClient, projectName, fields):
    issues = []
    i = 0
    chunkSize = 100
    while True:
        chunk = jiraClient.search_issues(f'project = {projectName}', startAt=i, 
                                         maxResults=chunkSize, fields=fields)
        i += chunkSize
        issues += chunk.iterable
        if i >= chunk.total:
            break
    return issues

In [4]:
import warnings
warnings.filterwarnings('ignore')

jiraIssuesFile = 'jiraIssues.csv'

if not(os.path.isfile(jiraIssuesFile)) :
    issues = getAllIssues(jira, 'ActiveMQ', ['id', 'resolution', 'status', 'issuetype', 'description'])
    
    with open(jiraIssuesFile, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['id', 'key', 'status', 'issuetype', 'resolution', 'description'])
        writer.writeheader()  
        
        for issue in issues:
            if issue.raw['fields']['issuetype']['name'] == 'Bug':
                
                record = {'id': issue.id, 'key': issue.key, 
                          'status': issue.raw['fields']['status']['name'], 
                          'issuetype': issue.raw['fields']['issuetype']['name'],
                          'resolution': "None" if (issue.raw['fields']['resolution'] is None) else issue.raw['fields']['resolution']['name'],
                          'description': issue.raw['fields']['description']
                         }
                writer.writerow(record)

In [5]:
commitDf = pd.read_csv(commitsFile, dtype={"hash": "string", "msg": "string", "insertions": "int"})
commitDf.head()

,hash,msg,insertions
0,40a7d3b6ac35d2ecb34e85fc3403d2e48e33874e,Moved the trunk code into the trunk sub direct...,194375
1,8f1763f078525b3cbfd30dd8389d3f61da56ac78,Moved the trunk code into the trunk sub direct...,899
2,262a5596d9300b7aded14d550cf8f5ee80d7ac0f,optimisation; if a JMS exception has already b...,6
3,1835e4536984abac76c6d4317a73f1a7e851e960,added some helper methods to make it easy to s...,28
4,e1cfbad4bcb9d3342da0b198798db120914a1976,added test case for using the BrokerService wi...,29


In [6]:
bugDf = pd.read_csv(jiraIssuesFile, dtype={"id": "string", "resolution": "string", "status": "string", 
                                           "issuetype": "string", "description": "string"})
bugDf = bugDf.loc[(bugDf['status'] == 'Resolved') | (bugDf['resolution'] == 'Fixed')]
bugDf.head()

,id,key,status,issuetype,resolution,description
4,13407920,AMQ-8408,Resolved,Bug,Fixed,<NA>
6,13404313,AMQ-8395,Resolved,Bug,Fixed,{noformat} AdvisoryBroker |...
7,13402290,AMQ-8389,Closed,Bug,Fixed,We’ve found the following broken URls on the A...
8,13401461,AMQ-8386,Resolved,Bug,Fixed,We’ve identified the following broken URLs on ...
17,13395929,AMQ-8357,Closed,Bug,Fixed,The download page [1] refers several times to ...


In [11]:
def getKeyMutations(bugKey):
    return [bugKey, bugKey.replace('-', '')]

def isMatchedWithKey(message, keys):
    for key in keys:
        splittedMessage = message.split(key,1)
        if len(splittedMessage) == 2 and (splittedMessage[1] == '' or (not splittedMessage[1][0].isdigit())):
            return True
    return False
        
def findExactKeys(relevantCommits, keys):
    relevantCommits['exactBugKey'] = relevantCommits['msg'].apply(lambda m: isMatchedWithKey(m, keys))
    relevantCommits = relevantCommits.loc[relevantCommits['exactBugKey'] == True]

In [12]:
ii = 0
excMergeCommits = commitDf.loc[commitDf['msg'].str.contains("Merge pull request") == False]

for index, bug in bugDf.iterrows():
    keyMutations = getKeyMutations(bug.key)
    relevantCommits = excMergeCommits.loc[excMergeCommits['msg'].str.contains(keyMutations[0]) | excMergeCommits['msg'].str.contains(keyMutations[1])]
    
    if not(relevantCommits.empty):
        findExactKeys(relevantCommits, keyMutations)
        ii += 1
print(ii)

639


In [26]:
for index, commit in commitDf.iterrows():
    if "AMQ-" not in commit.msg and ("AMQ" in commit.msg or "amq" in commit.msg):
        print(commit.msg)
        print("************************")

Bug: clean goal doesn't delete the amq's basedir generated target and xdocs directory.
Fix: Modified clean goal to delete these directories.

git-svn-id: https://svn.apache.org/repos/asf/incubator/activemq/trunk@378412 13f79535-47bb-0310-9956-ffa450edef68
************************
Added AMQ common test settings properties files

git-svn-id: https://svn.apache.org/repos/asf/incubator/activemq/trunk@418921 13f79535-47bb-0310-9956-ffa450edef68
************************
added isStoreValid() method - used by AMQStore to determine if references need to be rebuilt

git-svn-id: https://svn.apache.org/repos/asf/activemq/trunk@516407 13f79535-47bb-0310-9956-ffa450edef68
************************
Add properties to the AMQPersistenceAdaptorFactory
to set syncOnWrite(default=false) and persistentIndex(default=true) properties

git-svn-id: https://svn.apache.org/repos/asf/activemq/trunk@582801 13f79535-47bb-0310-9956-ffa450edef68
************************
Adding a modified version of the test case attac